In [3]:
# Install all dependencies
!pip install osmnx shapely opencv-python rasterio geopandas pyproj rtree


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.1/541.1 kB 32.3 MB/s eta 0:00:00


In [4]:
import cv2
import numpy as np
import osmnx as ox  # or OSMPythonTools
from shapely.geometry import Polygon, shape

def mask_overlaps_roads(image_path, mask_path, pixel_to_geo_fn):
    """
    image_path, mask_path: paths to PNG image and its corresponding mask (same dimensions)
    pixel_to_geo_fn: function mapping image pixel (x, y) to (lat, lon)
    Returns True if any masked area overlaps with roads in OSM.
    """

    img = cv2.imread(image_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if img.shape[:2] != mask.shape:
        raise ValueError("Image and mask must have the same dimensions")

    # Extract mask contours in pixel space
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return False

    # Convert pixel contours to lat/lon polygons
    poly_coords = []
    for pt in contours[0]:
        x, y = pt[0]
        lat, lon = pixel_to_geo_fn(x, y)
        poly_coords.append((lon, lat))  # Shapely expects (x, y) as (lon, lat)

    mask_polygon = Polygon(poly_coords)
    if mask_polygon.is_empty:
        return False

    # Determine bounding box for OSM query
    minx, miny, maxx, maxy = mask_polygon.bounds  # (lon_min, lat_min, lon_max, lat_max)

    # Query roads from OSM with OSMnx (common)
    G = ox.graph_from_bbox(maxy, miny, maxx, minx, network_type='drive')
    roads = ox.graph_to_gdfs(G, nodes=False, edges=True)

    # Check for any intersection
    for _, road in roads.iterrows():
        if mask_polygon.intersects(road.geometry):
            return True
    return False


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Example: Using the mask_overlaps_roads function

# Suppose our PNG image is a georeferenced satellite image
image_path = "/content/drive/MyDrive/P2 - Amazon ITU - PESU/Visual_data/GEE_Exports_all/id_1017.tif"
mask_path = "/content/drive/MyDrive/P2 - Amazon ITU - PESU/Visual_data/All_Masks_TIF/0_id_1017.tif"

# Let's say we know the bounding box (lat_min, lon_min, lat_max, lon_max) of the image
lat_min, lon_min = 37.7700, -122.4200  # bottom-left corner
lat_max, lon_max = 37.7800, -122.4100  # top-right corner

# And the image resolution
import cv2
h, w, _ = cv2.imread(image_path).shape

def pixel_to_geo_fn(x, y):
    """
    Convert pixel (x, y) to latitude/longitude based on known bounding box.
    - x is column index (0 to width)
    - y is row index (0 to height)
    """
    lon = lon_min + (x / w) * (lon_max - lon_min)
    lat = lat_max - (y / h) * (lat_max - lat_min)  # minus because y increases downward
    return lat, lon

# Now run the overlap check
result = mask_overlaps_roads(image_path, mask_path, pixel_to_geo_fn)

if result:
    print("✅ The mask overlaps with at least one road in OSM.")
else:
    print("❌ No overlap between the mask and roads in OSM.")


AttributeError: 'NoneType' object has no attribute 'shape'

In [9]:
import cv2
import numpy as np
import osmnx as ox
import rasterio
from shapely.geometry import Polygon

def mask_overlaps_roads_tif(image_path, mask_path, manual_bbox=None):
    """
    image_path: GeoTIFF or TIF satellite image
    mask_path: PNG/TIF binary mask (same dimensions as image)
    manual_bbox: (lat_min, lon_min, lat_max, lon_max) if no georeferencing is present
    Returns True if the mask overlaps with any road from OSM.
    """
    # Open image with rasterio
    with rasterio.open(image_path) as src:
        h, w = src.height, src.width
        transform = src.transform if src.transform else None
        crs = src.crs

    # Read mask (PNG with OpenCV, TIF with rasterio)
    if mask_path.lower().endswith(".tif") or mask_path.lower().endswith(".tiff"):
        with rasterio.open(mask_path) as msrc:
            mask = msrc.read(1)  # first band
    else:
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    if mask is None:
        raise FileNotFoundError(f"Could not read mask file: {mask_path}")
    if mask.shape != (h, w):
        raise ValueError("Image and mask dimensions must match")

    # Find mask contours
    contours, _ = cv2.findContours((mask > 0).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return False

    # Convert first contour to lat/lon polygon
    poly_coords = []
    for pt in contours[0]:
        x, y = pt[0]
        if transform and crs:
            lon, lat = rasterio.transform.xy(transform, y, x)
        elif manual_bbox:
            lat_min, lon_min, lat_max, lon_max = manual_bbox
            lon = lon_min + (x / w) * (lon_max - lon_min)
            lat = lat_max - (y / h) * (lat_max - lat_min)  # y increases downwards
        else:
            raise ValueError("No georeferencing found. Provide manual_bbox.")
        poly_coords.append((lon, lat))

    mask_polygon = Polygon(poly_coords)
    if mask_polygon.is_empty:
        return False

    # Get bounding box for OSM query
    minx, miny, maxx, maxy = mask_polygon.bounds

    # Query roads from OSM
    G = ox.graph_from_bbox(maxy, miny, maxx, minx, network_type='drive')
    roads = ox.graph_to_gdfs(G, nodes=False, edges=True)

    # Check for any intersection
    for _, road in roads.iterrows():
        if mask_polygon.intersects(road.geometry):
            return True
    return False


# ---------------- Example Usage ----------------
if __name__ == "__main__":
    image_path = "/content/drive/MyDrive/P2 - Amazon ITU - PESU/Visual_data/GEE_Exports_all/id_1000.tif"
    mask_path = "/content/drive/MyDrive/P2 - Amazon ITU - PESU/Visual_data/All_Masks_TIF/1_id_1000.tif"

    # If your TIF has no georeferencing, uncomment and fill manual bounding box:
    # manual_bbox = (37.7700, -122.4200, 37.7800, -122.4100)
    manual_bbox = None

    result = mask_overlaps_roads_tif(image_path, mask_path, manual_bbox)
    print("✅ Overlaps with road" if result else "❌ No road overlap")


/usr/local/lib/python3.11/dist-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


TypeError: graph_from_bbox() takes 1 positional argument but 4 positional arguments (and 1 keyword-only argument) were given

In [10]:


with rasterio.open("/content/drive/MyDrive/P2 - Amazon ITU - PESU/Visual_data/GEE_Exports_all/id_100.tif") as src:
    print("CRS:", src.crs)
    print("Transform:", src.transform)


CRS: EPSG:4326
Transform: | 0.00, 0.00,-63.53|
| 0.00,-0.00, 1.34|
| 0.00, 0.00, 1.00|


In [4]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 82.7 MB/s eta 0:00:00


In [5]:
import rasterio

In [6]:
image_path = "/content/drive/MyDrive/P2 - Amazon ITU - PESU/Visual_data/GEE_Exports_all/id_100.tif"

with rasterio.open(image_path) as src:
    print("CRS:", src.crs)  # Coordinate Reference System
    print("Transform:", src.transform)  # Pixel -> coordinates mapping
    print("Bounds:", src.bounds)  # (minX, minY, maxX, maxY)
    lon, lat = rasterio.transform.xy(src.transform, row, col)



CRS: EPSG:4326
Transform: | 0.00, 0.00,-63.53|
| 0.00,-0.00, 1.34|
| 0.00, 0.00, 1.00|
Bounds: BoundingBox(left=-63.53415695310886, bottom=1.2907892317513403, right=-63.488253042090356, top=1.3369626373550838)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 3.9 MB/s eta 0:00:00


In [9]:
import rasterio
import osmnx as ox
import geopandas as gpd
from pyproj import Transformer

def get_osm_features_from_image(image_path, manual_bbox=None):
    """
    Extracts roads, buildings, and water bodies from OSM for the area covered by an image.

    Parameters:
        image_path (str): Path to the GeoTIFF image.
        manual_bbox (tuple): Optional bounding box (lat_min, lon_min, lat_max, lon_max)
                             if image is not georeferenced.

    Returns:
        dict: {"roads": GeoDataFrame, "buildings": GeoDataFrame, "water": GeoDataFrame}
    """
    # 1. Read bounding box
    with rasterio.open(image_path) as src:
        bounds = src.bounds
        crs = src.crs

    # 2. Get bounding box in EPSG:4326
    if crs and crs.to_string() != "EPSG:4326":
        transformer = Transformer.from_crs(crs, "EPSG:4326", always_xy=True)
        min_lon, min_lat = transformer.transform(bounds.left, bounds.bottom)
        max_lon, max_lat = transformer.transform(bounds.right, bounds.top)
    elif crs:
        min_lon, min_lat, max_lon, max_lat = bounds.left, bounds.bottom, bounds.right, bounds.top
    elif manual_bbox:
        lat_min, lon_min, lat_max, lon_max = manual_bbox
        min_lon, min_lat, max_lon, max_lat = lon_min, lat_min, lon_max, lat_max
    else:
        raise ValueError("Image is not georeferenced. Provide manual_bbox.")

    # 3. Query OSM for each type of feature
    features = {}

    # Roads
    G = ox.graph_from_bbox(max_lat, min_lat, max_lon, min_lon, network_type='drive')
    features["roads"] = ox.graph_to_gdfs(G, nodes=False, edges=True)

    # Buildings
    tags_buildings = {"building": True}
    features["buildings"] = ox.geometries_from_bbox(max_lat, min_lat, max_lon, min_lon, tags_buildings)

    # Water bodies
    tags_water = {"natural": "water"}
    features["water"] = ox.geometries_from_bbox(max_lat, min_lat, max_lon, min_lon, tags_water)

    return features


# ---------------- Example usage ----------------
if __name__ == "__main__":
    image_path = "/content/drive/MyDrive/P2 - Amazon ITU - PESU/Visual_data/GEE_Exports_all/id_1660.tif"

    # Optional: If not georeferenced
    # manual_bbox = (37.7700, -122.4200, 37.7800, -122.4100)
    manual_bbox = None

    osm_data = get_osm_features_from_image(image_path, manual_bbox)

    print("Roads:", osm_data["roads"].shape)
    print("Buildings:", osm_data["buildings"].shape)
    print("Water bodies:", osm_data["water"].shape)

    # Example: plot roads
    osm_data["roads"].plot()


TypeError: graph_from_bbox() takes 1 positional argument but 4 positional arguments (and 1 keyword-only argument) were given

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import rasterio
from rasterio.warp import transform_bounds

def get_latlon_bbox(tif_path):
    with rasterio.open(tif_path) as src:
        # Original bounding box in the image's CRS
        bounds = src.bounds
        crs = src.crs

        # Convert bounding box to WGS84 (lat/lon)
        latlon_bounds = transform_bounds(crs, "EPSG:4326",
                                         bounds.left, bounds.bottom,
                                         bounds.right, bounds.top)
        min_lon, min_lat, max_lon, max_lat = latlon_bounds
        return min_lat, max_lat, min_lon, max_lon

# Example usage:
tif_path = "/content/drive/MyDrive/P2 - Amazon ITU - PESU/Visual_data/GEE_Exports_all/id_1660.tif"
min_lat, max_lat, min_lon, max_lon = get_latlon_bbox(tif_path)
print("Bounding box in lat/lon:")
print(f"min_lat: {min_lat}, max_lat: {max_lat}")
print(f"min_lon: {min_lon}, max_lon: {max_lon}")


Bounding box in lat/lon:
min_lat: 2.667457404664507, max_lat: 2.7136308102682505
min_lon: -61.540346180005585, max_lon: -61.49444226898708


In [32]:
import osmnx as ox

def get_osm_data(max_lat, min_lat, max_lon, min_lon):
    # Roads
    G = ox.graph_from_bbox((max_lat, min_lat, max_lon, min_lon), network_type='drive')
    #G = ox.graph_from_bbox(north=max_lat, south=min_lat, east=max_lon, west=min_lon)
    roads = ox.graph_to_gdfs(G, nodes=False, edges=True)



    # # Buildings
    # buildings = ox.geometries_from_bbox(max_lat, min_lat, max_lon, min_lon, tags={"building": True})

    # # Water bodies
    # water = ox.geometries_from_bbox(max_lat, min_lat, max_lon, min_lon, tags={"natural": "water"})

    return roads #, buildings, water

# Example usage (replace with values from get_latlon_bbox)
min_lat = 28.613500
max_lat = 28.622500
min_lon = 77.208000
max_lon = 77.217000



# tif_path = "/content/drive/MyDrive/P2 - Amazon ITU - PESU/Visual_data/GEE_Exports_all/id_1660.tif"
# min_lat, max_lat, min_lon, max_lon = get_latlon_bbox(tif_path)
roads = get_osm_data(max_lat, min_lat, max_lon, min_lon)

print("Roads:", roads[['name', 'highway']].head())
# print("Buildings:", buildings[['building']].head())
# print("Water:", water.head())


/usr/local/lib/python3.11/dist-packages/osmnx/_overpass.py:271: UserWarning: This area is 6,329 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


KeyboardInterrupt: 

In [38]:
import osmnx as ox

north = 23.5045
south = 23.4955
east  = 78.004135
west  = 77.995865
print("north:", north)
print("south:", south)
print("east:", east)
print("west:", west)

approx_area_km2 = abs(north - south) * 111 * abs(east - west) * 111
print("Approx area km²:", approx_area_km2)


G = ox.graph_from_bbox((north, south, east, west), network_type="drive")
ox.plot_graph(G)


north: 23.5045
south: 23.4955
east: 78.004135
west: 77.995865
Approx area km²: 0.9170520300011673


/usr/local/lib/python3.11/dist-packages/osmnx/_overpass.py:271: UserWarning: This area is 8,211 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


KeyboardInterrupt: 